In [2]:
import logging
import glob
import argparse
from gensim import models
from gensim import matutils
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time
from nltk import bigrams
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora
from stop_words import get_stop_words
from collections import Counter
from itertools import product, compress
from collections import defaultdict
import math
import pandas as pd
from nltk.stem.snowball import SnowballStemmer

## Keyword

In [4]:
keyword = pd.read_csv("keyword.csv", names = ['keyword', 'score'])

In [5]:
keyword = keyword[keyword['score'] > 0.95]

In [6]:
keyword

,keyword,score
0,sleeper_sofa,0.996589
1,lower_level,0.996525
2,burke_gilman_trail,0.996525
3,university_district,0.996431
4,pacific_science_center,0.996431
5,gas_stove,0.996431
6,gas_fireplace,0.996431
7,capital_hill,0.996431
8,vacation_rental,0.996182
9,south_lake_union,0.996182


In [10]:
keyword['keyword'] = keyword['keyword'].apply(lambda x : x.replace("_", " "))

In [111]:
keyword

,keyword,score
0,sleeper sofa,0.996589
1,lower level,0.996525
2,burke gilman trail,0.996525
3,university district,0.996431
4,pacific science center,0.996431
5,gas stove,0.996431
6,gas fireplace,0.996431
7,capital hill,0.996431
8,vacation rental,0.996182
9,south lake union,0.996182


## Listing

In [46]:
listing = pd.read_csv("seattle-airbnb-open-data/listings.csv")

In [13]:
listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
id                                  3818 non-null int64
listing_url                         3818 non-null object
scrape_id                           3818 non-null int64
last_scraped                        3818 non-null object
name                                3818 non-null object
summary                             3641 non-null object
space                               3249 non-null object
description                         3818 non-null object
experiences_offered                 3818 non-null object
neighborhood_overview               2786 non-null object
notes                               2212 non-null object
transit                             2884 non-null object
thumbnail_url                       3498 non-null object
medium_url                          3498 non-null object
picture_url                         3818 non-null object
xl_picture_url                      3498

In [47]:
listing = listing.drop(['scrape_id', 'last_scraped', 'host_response_time', 'host_response_rate', 'host_acceptance_rate',\
             'host_is_superhost', 'host_listings_count', 'host_total_listings_count', 'host_verifications',\
             'host_identity_verified', 'latitude', 'longitude', 'is_location_exact', 'has_availability',\
             'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped',\
             'requires_license', 'license', 'jurisdiction_names', 'cancellation_policy', 'require_guest_profile_picture',\
             'require_guest_phone_verification', 'calculated_host_listings_count', 'instant_bookable', 'calculated_host_listings_count',\
                        'first_review', 'last_review', 'review_scores_accuracy', 'number_of_reviews'], axis = 1)

In [15]:
listing.head()

,id,listing_url,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,...,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,4.07
1,953595,https://www.airbnb.com/rooms/953595,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",...,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,...,2015-09-03,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
4,278830,https://www.airbnb.com/rooms/278830,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,Belltown,The nearest public transit bus (D Line) is 2 b...,...,2015-10-24,92.0,9.0,9.0,10.0,10.0,9.0,9.0,f,0.89


In [25]:
listing['space'].to_csv('listDes.txt', index = False, sep = '\n')

In [33]:
with open('listDes.txt', 'r') as f:
    reviews = f.read().splitlines()

In [101]:
import string
def preposs(line):
    split = line.split()
    stemmer = SnowballStemmer("english")
    split = [stemmer.stem(w) for w in split]
    return ' '.join(split)
    
    

In [103]:
vectorizer = CountVectorizer(ngram_range = (2,3), preprocessor = preposs, 
                            token_pattern = r'\w+',
                            stop_words = 'english')
X = vectorizer.fit_transform(reviews)

In [104]:
terms = vectorizer.get_feature_names()

In [131]:
Xarrary = X.toarray()

In [151]:
keywordlist = []
listid = []
for i in range(len(terms)):
    if terms[i] in keyword['keyword'].tolist():
        keywordlist.append(terms[i])
        lists = Xarrary[:,i]
        listids = [i for i, x in enumerate(lists) if x == 1]
        listid.append(listids)

In [155]:
pd.DataFrame({"keyword": keywordlist, "listid": listid}).to_csv('keywordToListId.csv', index = False)

In [158]:
df = pd.DataFrame({"keyword": keywordlist, "listid": listid})

In [161]:
df

,keyword,listid
0,air conditioning,"[101, 134, 309, 506, 516, 528, 682, 731, 823, ..."
1,air mattress,"[25, 86, 147, 149, 154, 165, 166, 207, 210, 28..."
2,alki beach,"[1722, 1735, 1742, 1752, 1756, 1759, 1763, 176..."
3,amazon prime,"[118, 371, 460, 607, 608, 944, 1346, 1388, 147..."
4,art galleries,"[1209, 1270, 1278, 1293, 1329, 1437, 1694, 189..."
5,basement apartment,"[705, 924, 1724, 1870, 1875, 1895, 1947, 2157,..."
6,basic cable,"[0, 434, 687, 689, 718, 725, 726, 733, 758, 76..."
7,beacon hill,"[1774, 1948, 2066, 2072, 2076, 2083, 2085, 208..."
8,blu ray,"[45, 163, 889, 1080, 1318, 1471, 1516, 1550, 1..."
9,board games,"[133, 523, 1858, 2123, 2414, 3035, 3473, 3809]"


In [162]:
df['len'] = df['listid'].apply(lambda x: len(x))

In [167]:
df.sort('len', ascending=False).to_csv('keywordToListId.csv', index = False)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


#### pd.DataFrame({"keyword": keywordlist, "listid": listid})

## Reviews

In [13]:
reviews = pd.read_csv('seattle-airbnb-open-data/reviews.csv')

In [84]:
reviews['comments'].to_csv('review.txt', index = False, sep = '\n')

In [52]:
sentiment = pd.read_csv('Sentiment_Analysis/sentiment.txt', sep="\t", names = ['text', 'sentiment_score', 'sentiment'])

In [53]:
sentiment.head()

,text,sentiment_score,sentiment
0,Cute and cozy place. Perfect location to every...,3,Positive
1,Kelly has a great room in a very central locat...,2,Neutral
2,"Very spacious apartment, and in a great neighb...",1,Negative
3,Close to Seattle Center and all it has to offe...,1,Negative
4,Kelly was a great host and very accommodating ...,3,Positive


In [54]:
sentiment['listing_id'] = reviews['listing_id']

In [56]:
score_mean = pd.DataFrame(sentiment.groupby('listing_id').sentiment_score.mean())
score_mean = score_mean.reset_index()

In [57]:
listingToSentiment = pd.merge(listing, score_mean, how = 'left', left_on='id', right_on='listing_id')

In [59]:
listingToSentiment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3818 entries, 0 to 3817
Data columns (total 63 columns):
id                              3818 non-null int64
listing_url                     3818 non-null object
name                            3818 non-null object
summary                         3641 non-null object
space                           3249 non-null object
description                     3818 non-null object
experiences_offered             3818 non-null object
neighborhood_overview           2786 non-null object
notes                           2212 non-null object
transit                         2884 non-null object
thumbnail_url                   3498 non-null object
medium_url                      3498 non-null object
picture_url                     3818 non-null object
xl_picture_url                  3498 non-null object
host_id                         3818 non-null int64
host_url                        3818 non-null object
host_name                       3816 non-null o

In [73]:
listingToSentiment.drop(['listing_id', 'experiences_offered', 'description', 'notes', 'maximum_nights', 
                        'calendar_updated', 'host_id', 'host_url', 'host_location', 'host_has_profile_pic',
                        'host_neighbourhood', 'host_since', 'host_location', 'medium_url', 'neighbourhood_group_cleansed',
                        'smart_location', 'country_code', 'country', 'guests_included', 'picture_url',
                        'xl_picture_url', 'reviews_per_month', 'market', 'host_picture_url', 'weekly_price',
                        'monthly_price', 'minimum_nights', 'security_deposit', 'extra_people',
                        'neighbourhood_cleansed', 'neighborhood_overview', 'square_feet',
                        'review_scores_rating', 'review_scores_cleanliness', 'review_scores_checkin',
                        'review_scores_communication', 'review_scores_location', 'review_scores_value',
                        'amenities', 'summary'], axis = 1).to_csv('listingToSentiment.csv', index = False)